# using huggingface and distilbert 

In [44]:
import pandas as pd 
df = pd.read_csv('dataset/spamdetection.csv',sep='\t',names=['label','message']).dropna()
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [45]:
df.shape # 5574 rows and 2 columns

(5571, 2)

In [46]:
x = list(df['message'])
y = list(df['label'])

In [47]:
y = list(pd.get_dummies(y,drop_first=True) ['spam']) #take spam column

In [ ]:
y

#  train test split

In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [49]:
from transformers import DistilBertTokenizerFast # differnt architecture use diffenet tokenizier 
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased') # load from this model

In [50]:
#row must be nullfree that means dropna() must be used when load dataset

train_encoding = tokenizer(X_train,truncation=True, padding=True) # padding keep sentence same length 
test_encoding = tokenizer(X_test,truncation=True, padding=True)   

# Convert this encoding into dataset objects

In [51]:
import tensorflow as tf # as we used tensorflow so we need to use tensor

In [55]:
train_dataset = tf.data.Dataset.from_tensors((dict(train_encoding),y_train)) #data object
train_dataset

<TensorDataset shapes: ({input_ids: (4456, 238), attention_mask: (4456, 238)}, (4456,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>